In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
#Read in Data
train = pd.read_csv('dataFiles/application_train.csv')
test = pd.read_csv('dataFiles/application_test.csv')
bureau_data = pd.read_csv('dataFiles/bureau.csv')
prev_app_data = pd.read_csv('dataFiles/previous_application.csv')

In [3]:
train_subset = train[['SK_ID_CURR', 'TARGET', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY']]

In [4]:
annuity_avg = np.mean(train_subset.AMT_ANNUITY)

In [5]:
train_subset.AMT_ANNUITY = train_subset.AMT_ANNUITY.fillna(annuity_avg)

/Users/MikeTodisco/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [6]:
X, y = train_subset.drop(['SK_ID_CURR','TARGET'], axis = 1), train_subset.TARGET

In [7]:
model = RandomForestClassifier()

In [8]:
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [9]:
model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [10]:
test_subset = test[['SK_ID_CURR', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY']]

In [11]:
test_subset.AMT_ANNUITY = test_subset.AMT_ANNUITY.fillna(np.mean(test_subset.AMT_ANNUITY))

In [12]:
test_X = test_subset.drop('SK_ID_CURR', axis = 1)

In [13]:
test_y = pd.DataFrame(model.predict_proba(test_X))

In [14]:
submission_df = pd.concat([test.SK_ID_CURR, test_y], axis=1).drop(0, axis = 1)

In [15]:
submission_df.columns = ['SK_ID_CURR', 'Target']

In [16]:
submission_df.to_csv('first_submission.csv', index = False)

In [17]:
############################## All Numeric Non-Null Columns

In [18]:
numeric_train = train.select_dtypes(exclude='object')

In [19]:
numeric_train_subset = numeric_train.dropna(axis = 1)

In [20]:
X, y = numeric_train_subset.drop(['SK_ID_CURR','TARGET'], axis = 1), numeric_train_subset.TARGET

In [21]:
model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [22]:
test_X2 = test[X.columns]

In [23]:
test_y2 = pd.DataFrame(model.predict_proba(test_X2))

In [24]:
submission_df2 = pd.concat([test.SK_ID_CURR, test_y2], axis=1).drop(0, axis = 1)

In [25]:
submission_df2.columns = ['SK_ID_CURR', 'Target']

In [26]:
submission_df2.to_csv('2nd_submission.csv', index = False)

In [27]:
##### catboost

In [28]:
from catboost import CatBoostClassifier

In [29]:
cat_model = CatBoostClassifier(iterations=100,
    learning_rate=0.1)

In [30]:
cat_model.fit(X, y)

0:	learn: 0.5945277	total: 238ms	remaining: 23.6s
1:	learn: 0.5196972	total: 412ms	remaining: 20.2s
2:	learn: 0.4621204	total: 626ms	remaining: 20.2s
3:	learn: 0.4191080	total: 816ms	remaining: 19.6s
4:	learn: 0.3858453	total: 1.03s	remaining: 19.5s
5:	learn: 0.3607293	total: 1.22s	remaining: 19.1s
6:	learn: 0.3410788	total: 1.42s	remaining: 18.8s
7:	learn: 0.3263074	total: 1.59s	remaining: 18.3s
8:	learn: 0.3146036	total: 1.81s	remaining: 18.3s
9:	learn: 0.3054697	total: 1.99s	remaining: 17.9s
10:	learn: 0.2982259	total: 2.2s	remaining: 17.8s
11:	learn: 0.2926161	total: 2.37s	remaining: 17.4s
12:	learn: 0.2881357	total: 2.58s	remaining: 17.3s
13:	learn: 0.2846350	total: 2.76s	remaining: 17s
14:	learn: 0.2818526	total: 2.97s	remaining: 16.9s
15:	learn: 0.2796237	total: 3.15s	remaining: 16.6s
16:	learn: 0.2777417	total: 3.36s	remaining: 16.4s
17:	learn: 0.2762495	total: 3.54s	remaining: 16.1s
18:	learn: 0.2750969	total: 3.74s	remaining: 16s
19:	learn: 0.2741230	total: 3.93s	remaining: 1

In [31]:
test_y_cat = pd.DataFrame(cat_model.predict_proba(test_X2))
submission_cat = pd.concat([test.SK_ID_CURR, test_y_cat], axis=1).drop(0, axis = 1)
submission_cat.columns = ['SK_ID_CURR', 'Target']
submission_cat.to_csv('submission_cat.csv', index = False)

In [32]:
##### EDA

In [33]:
import matplotlib.pyplot as plt

In [34]:
train_subset = train_subset.loc[train_subset.AMT_INCOME_TOTAL <= 100000000]

In [35]:
numeric_train.groupby('TARGET').median()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
TARGET,,,,,,,,,,,,,,,,,,,,,
0,278362.5,0.0,148500.0,517788.0,24876.0,450000.0,0.018850,-15877.0,-1235.0,-4544.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,276291.0,0.0,135000.0,497520.0,25263.0,450000.0,0.018634,-14282.0,-1034.0,-4056.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [36]:
numeric_train.FLAG_DOCUMENT_4.value_counts()

0    307486
1        25
Name: FLAG_DOCUMENT_4, dtype: int64

In [37]:
from sklearn.metrics import confusion_matrix

In [38]:
confusion_matrix(numeric_train.FLAG_DOCUMENT_4, numeric_train.TARGET)

array([[282661,  24825],
       [    25,      0]])

In [39]:
numeric_train.FLAG_DOCUMENT_4.value_counts()

0    307486
1        25
Name: FLAG_DOCUMENT_4, dtype: int64

In [40]:
numeric_train.TARGET.value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [41]:
##### Exploring Bureau Data

In [42]:
bureau_data.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [43]:
bureau_data_grouped = bureau_data.groupby('SK_ID_CURR', as_index=False).agg({'SK_ID_BUREAU': 'count', 'AMT_CREDIT_MAX_OVERDUE':sum,\
                                                      'AMT_CREDIT_SUM_OVERDUE': sum})

In [44]:
# Joining Bureau Grouped Data with Train Data

merged_data = train.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [45]:
merged_data = merged_data[['SK_ID_CURR','TARGET','NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','NAME_TYPE_SUITE',\
            'NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','REGION_POPULATION_RELATIVE','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH',\
            'OWN_CAR_AGE','FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL','CNT_FAM_MEMBERS','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY',\
            'WEEKDAY_APPR_PROCESS_START','EMERGENCYSTATE_MODE','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8',\
            'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18',\
            'FLAG_DOCUMENT_21','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21','AMT_CREDIT_MAX_OVERDUE','AMT_CREDIT_SUM_OVERDUE']]


In [46]:
merged_data_dummies = pd.get_dummies(merged_data)

In [47]:
def fill_missing_avg(df, col):
    df[col] = df[col].fillna(np.mean(df[col]))

In [48]:
fill_missing_avg(merged_data_dummies, 'AMT_ANNUITY')
fill_missing_avg(merged_data_dummies, 'AMT_GOODS_PRICE')
fill_missing_avg(merged_data_dummies, 'OWN_CAR_AGE')
fill_missing_avg(merged_data_dummies, 'CNT_FAM_MEMBERS')
fill_missing_avg(merged_data_dummies, 'AMT_CREDIT_MAX_OVERDUE')
fill_missing_avg(merged_data_dummies, 'AMT_CREDIT_MAX_OVERDUE')
fill_missing_avg(merged_data_dummies, 'AMT_CREDIT_SUM_OVERDUE')

In [49]:
X, y = merged_data_dummies.drop(['SK_ID_CURR','TARGET','CODE_GENDER_XNA','NAME_FAMILY_STATUS_Unknown', 'NAME_INCOME_TYPE_Maternity leave'], axis = 1), merged_data_dummies.TARGET

In [50]:
cat_model.fit(X, y)

0:	learn: 0.5939008	total: 323ms	remaining: 31.9s
1:	learn: 0.5178417	total: 660ms	remaining: 32.3s
2:	learn: 0.4602223	total: 975ms	remaining: 31.5s
3:	learn: 0.4170903	total: 1.28s	remaining: 30.8s
4:	learn: 0.3836631	total: 1.62s	remaining: 30.8s
5:	learn: 0.3581166	total: 1.94s	remaining: 30.5s
6:	learn: 0.3385851	total: 2.27s	remaining: 30.2s
7:	learn: 0.3230295	total: 2.6s	remaining: 29.9s
8:	learn: 0.3109420	total: 2.91s	remaining: 29.4s
9:	learn: 0.3017065	total: 3.23s	remaining: 29s
10:	learn: 0.2946315	total: 3.54s	remaining: 28.7s
11:	learn: 0.2889119	total: 3.91s	remaining: 28.7s
12:	learn: 0.2842613	total: 4.29s	remaining: 28.7s
13:	learn: 0.2805721	total: 4.61s	remaining: 28.3s
14:	learn: 0.2776368	total: 4.93s	remaining: 27.9s
15:	learn: 0.2753686	total: 5.25s	remaining: 27.6s
16:	learn: 0.2734282	total: 5.6s	remaining: 27.3s
17:	learn: 0.2718833	total: 6.15s	remaining: 28s
18:	learn: 0.2706104	total: 6.61s	remaining: 28.2s
19:	learn: 0.2694794	total: 6.93s	remaining: 27

In [51]:
merged_test = test.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [52]:
merged_test = merged_test[['SK_ID_CURR','NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','NAME_TYPE_SUITE',\
            'NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','REGION_POPULATION_RELATIVE','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH',\
            'OWN_CAR_AGE','FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL','CNT_FAM_MEMBERS','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY',\
            'WEEKDAY_APPR_PROCESS_START','EMERGENCYSTATE_MODE','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8',\
            'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18',\
            'FLAG_DOCUMENT_21','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21','AMT_CREDIT_MAX_OVERDUE','AMT_CREDIT_SUM_OVERDUE']]

In [53]:
merged_test_dummies = pd.get_dummies(merged_test)

In [54]:
fill_missing_avg(merged_test_dummies, 'AMT_ANNUITY')
fill_missing_avg(merged_test_dummies, 'OWN_CAR_AGE')
fill_missing_avg(merged_test_dummies, 'AMT_CREDIT_MAX_OVERDUE')
fill_missing_avg(merged_test_dummies, 'AMT_CREDIT_SUM_OVERDUE')

In [55]:
IDs = merged_test_dummies['SK_ID_CURR']

In [56]:
merged_test_dummies = merged_test_dummies.drop('SK_ID_CURR', axis = 1)

In [57]:
test_y_cat = pd.DataFrame(cat_model.predict_proba(merged_test_dummies))
submission_cat = pd.concat([IDs, test_y_cat], axis=1).drop(0, axis = 1)
submission_cat.columns = ['SK_ID_CURR', 'Target']
submission_cat.to_csv('submission_cat2.csv', index = False)

In [58]:
##### Naive Bayes

In [59]:
from sklearn.naive_bayes import BernoulliNB

In [60]:
nb_model = BernoulliNB()

In [61]:
merged_data_dummies['DAYS_BIRTH'] = merged_data_dummies['DAYS_BIRTH'] * -1

/Users/MikeTodisco/anaconda2/lib/python2.7/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [62]:
merged_data_dummies['DAYS_REGISTRATION'] = merged_data_dummies['DAYS_REGISTRATION'].apply(lambda x: x * -1 if x < 0 else x)
merged_data_dummies['DAYS_EMPLOYED'] = merged_data_dummies['DAYS_EMPLOYED'].apply(lambda x: x * -1 if x < 0 else x)
merged_data_dummies['DAYS_ID_PUBLISH'] = merged_data_dummies['DAYS_ID_PUBLISH'].apply(lambda x: x * -1 if x < 0 else x)

In [63]:
X, y = merged_data_dummies.drop(['SK_ID_CURR','TARGET','CODE_GENDER_XNA','NAME_FAMILY_STATUS_Unknown', 'NAME_INCOME_TYPE_Maternity leave'], axis = 1), merged_data_dummies.TARGET

In [64]:
nb_model.fit(X, y)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [65]:
merged_test_dummies['DAYS_BIRTH'] = merged_test_dummies['DAYS_BIRTH'].apply(lambda x: x * -1 if x < 0 else x)
merged_test_dummies['DAYS_REGISTRATION'] = merged_test_dummies['DAYS_REGISTRATION'].apply(lambda x: x * -1 if x < 0 else x)
merged_test_dummies['DAYS_EMPLOYED'] = merged_test_dummies['DAYS_EMPLOYED'].apply(lambda x: x * -1 if x < 0 else x)
merged_test_dummies['DAYS_ID_PUBLISH'] = merged_test_dummies['DAYS_ID_PUBLISH'].apply(lambda x: x * -1 if x < 0 else x)
merged_test_dummies['REGION_RATING_CLIENT_W_CITY'] = merged_test_dummies['REGION_RATING_CLIENT_W_CITY'].apply(lambda x: x * -1 if x < 0 else x)

In [66]:
test_y_nb = pd.DataFrame(nb_model.predict_proba(merged_test_dummies))
submission_nb = pd.concat([IDs, test_y_nb], axis=1).drop(0, axis = 1)
submission_nb.columns = ['SK_ID_CURR', 'Target']
submission_nb.to_csv('submission_nb1.csv', index = False)

In [67]:
### Random Forest Classifier

In [68]:
from sklearn.ensemble import RandomForestClassifier

In [79]:
rf_model = RandomForestClassifier(n_estimators =100, 
                                       random_state = 1, 
                                       max_depth = 20,
                                       criterion='entropy')

In [80]:
rf_model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [81]:
test_y_rf = pd.DataFrame(rf_model.predict_proba(merged_test_dummies))
submission_rf = pd.concat([IDs, test_y_rf], axis=1).drop(0, axis = 1)
submission_rf.columns = ['SK_ID_CURR', 'Target']
submission_rf.to_csv('submission_rf1.csv', index = False)

In [90]:
pd.DataFrame(sorted(zip(rf_model.feature_importances_, X.columns), reverse=True), \
             columns = ['Importance','Metric']).head(15)

,Importance,Metric
0,0.080355,DAYS_BIRTH
1,0.073838,DAYS_EMPLOYED
2,0.072021,DAYS_ID_PUBLISH
3,0.069197,DAYS_REGISTRATION
4,0.065358,AMT_ANNUITY
5,0.062652,AMT_CREDIT
6,0.055953,AMT_GOODS_PRICE
7,0.051240,AMT_INCOME_TOTAL
8,0.051185,REGION_POPULATION_RELATIVE
9,0.038621,AMT_CREDIT_MAX_OVERDUE
